In [ ]:
import os
import shutil
import datetime
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


In [ ]:
### Set Browser
#!apt-get update 
#!apt install chromium-chromedriver

chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver.get('https://www.linkedin.com/login')

In [ ]:
# Find the username/
username=''
password=''

uname = driver.find_element("id", "username") 
uname.send_keys(username)

# Find the password input field and send the password to the input field.
pword = driver.find_element("id", "password") 
pword.send_keys(password)
time.sleep(3)

driver.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button').click()


In [ ]:
### Go to Job Search
driver.find_element(By.XPATH,'//*[@id="global-nav"]/div/nav/ul/li[3]/a').click()
time.sleep(5)


In [ ]:
#search Jobs : entry level data analyst easy apply job adds in Ireland
driver.get('https://www.linkedin.com/jobs/search/?currentJobId=3135481518&f_AL=true&f_E=2&f_TPR=r604800&geoId=104738515&keywords=data%20analyst&location=Ireland')
time.sleep(10)



In [ ]:
links=[]
for page in range(2,5):
    time.sleep(2)
    jobs_block = driver.find_element(By.CLASS_NAME,'scaffold-layout__list-container')
    #jobs_list= jobs_block.find_elements(By.CSS_SELECTOR, '.scaffold-layout__list-container-item')
    all_links = driver.find_elements(By.TAG_NAME,'a')
    for a in all_links:
        if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view/") and a.get_attribute('href') not in links: 
            print(a.get_attribute('href'))
            links.append(a.get_attribute('href'))
        else:
            pass
        # scroll down for each job element
    driver.execute_script("arguments[0].scrollIntoView();", jobs_block)
    
    print(f'Collecting the links in the page: {page-1}')
    # go to next page:
    try:
        driver.find_element(By.XPATH,f"//button[@aria-label='Page {page}']").click()
    except:
        page=6
        pass
time.sleep(3)

In [ ]:
print(links)

In [ ]:
# Create empty lists to store information
job_titles = []
company_names = []
company_locations = []
work_methods = []
post_dates = []
work_times = [] 
job_desc = []

i = 0
j = 1
# Visit each link one by one to scrape the information
print('Visiting the links and collecting information just started.')
for i in range(len(links)):
    try:
        driver.get(links[i])
        i=i+1
        time.sleep(2)
        # Click See more.
        driver.find_element(By.XPATH,'//*[@id="ember32"]').click()
        time.sleep(2)
    except:
        pass
    
    # Find the general information of the job offer
    # Used try, except in case there is some missing information for some of the job offers
    contents = driver.find_elements(By.CLASS_NAME,'p5')
    for content in contents:
        try:
            job_titles.append(content.find_element(By.TAG_NAME,"h1").text)
            company_names.append(content.find_element(By.CLASS_NAME,"jobs-unified-top-card__company-name").text)
            company_locations.append(content.find_element(By.CLASS_NAME,"jobs-unified-top-card__bullet").text)
            work_methods.append(content.find_element(By.CLASS_NAME,"jobs-unified-top-card__workplace-type").text)
            post_dates.append(content.find_element(By.CLASS_NAME,"jobs-unified-top-card__posted-date").text)
            work_times.append(content.find_element(By.CLASS_NAME,"jobs-unified-top-card__job-insight").text)
            print(f'Scraping the Job Offer {j} DONE.')
            j+= 1
            
        except:
            pass
        time.sleep(2)
        
        # Scraping the job description
    job_description = driver.find_elements(By.CLASS_NAME,'jobs-description__content')
    for description in job_description:
        job_text = description.find_element(By.CLASS_NAME,"jobs-box__html-content").text
        job_desc.append(job_text)
        print(f'Scraping the Job Offer {j}')
        time.sleep(2)

In [ ]:
df = pd.DataFrame(list(zip(job_titles,company_names,
                    company_locations,work_methods,
                    post_dates,work_times)),
                    columns =['job_title', 'company_name',
                           'company_location','work_method',
                           'post_date','work_time'])


In [ ]:
df.head(10)

In [ ]:
driver.quit()

In [ ]:
#driver.quit()